In [13]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna
import json
from sklearn.metrics import mean_absolute_error

In [14]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [15]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2020-05-01') & (df.index <= '2020-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_trainnew_a, X_test_new_a, y_train_new_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

X_train_loc_a, X_test_loc_a, y_train_loc_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"], y_train_a], axis=1))
X_train_loc_b, X_test_loc_b, y_train_loc_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"], y_train_b], axis=1))
X_train_loc_c, X_test_loc_c, y_train_loc_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"], y_train_c], axis=1))


X_train_new = pd.concat([X_train_loc_a, X_train_loc_b, X_train_loc_c])
X_test_new = pd.concat([X_test_loc_a, X_test_loc_b, X_test_loc_c])
y_train_new = pd.concat([y_train_loc_a, y_train_loc_b, y_train_loc_c])
y_test = pd.concat([y_test_a, y_test_b, y_test_c])

In [16]:
#Create Pools
train_pool = Pool(X_train_new, y_train_new, cat_features=["location"])
test_pool = Pool(X_test_new, cat_features=["location"]) 

In [ ]:
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 4000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function": "LogCosh"
    }

    catboost_model_val = CatBoostRegressor(**params, verbose=200)
    catboost_model_val.fit(train_pool)
    pred = pd.DataFrame(catboost_model_val.predict(test_pool))
    MAE = mean_absolute_error(y_test, pred)

    return MAE
    
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train_new, y_train_new), n_trials=70)

[I 2023-11-09 13:56:52,244] A new study created in memory with name: no-name-8bc03a70-1559-4464-8e5d-b0ec493be2e4


0:	learn: 260.7752120	total: 119ms	remaining: 54.1s
200:	learn: 116.4828172	total: 24.9s	remaining: 31.4s
400:	learn: 88.1217344	total: 50.5s	remaining: 6.67s
453:	learn: 84.8727154	total: 57.4s	remaining: 0us


[I 2023-11-09 13:57:50,050] Trial 0 finished with value: 163.70377359809018 and parameters: {'iterations': 454, 'learning_rate': 0.007609586094904204, 'depth': 13, 'min_data_in_leaf': 5, 'l2_leaf_reg': 6, 'bagging_temperature': 0.9988277155857881, 'random_strength': 0.9478905708358121, 'border_count': 283, 'rsm': 0.10496405748175507}. Best is trial 0 with value: 163.70377359809018.


0:	learn: 259.4343923	total: 92ms	remaining: 5m 36s
200:	learn: 105.1125244	total: 21.1s	remaining: 6m 2s
400:	learn: 95.6918542	total: 43s	remaining: 5m 49s
600:	learn: 93.2506608	total: 1m 5s	remaining: 5m 32s
800:	learn: 91.8603610	total: 1m 28s	remaining: 5m 15s
1000:	learn: 91.0124727	total: 1m 51s	remaining: 4m 55s
1200:	learn: 90.1170011	total: 2m 14s	remaining: 4m 35s
1600:	learn: 88.9524908	total: 3m 1s	remaining: 3m 53s
1800:	learn: 88.4305405	total: 3m 25s	remaining: 3m 32s
2000:	learn: 88.0186277	total: 3m 49s	remaining: 3m 10s
2200:	learn: 87.6068437	total: 4m 13s	remaining: 2m 48s
2400:	learn: 87.2916574	total: 4m 38s	remaining: 2m 25s
2600:	learn: 86.9683005	total: 5m 1s	remaining: 2m 2s
2800:	learn: 86.6590867	total: 5m 26s	remaining: 1m 40s
3000:	learn: 86.4805106	total: 5m 51s	remaining: 1m 17s
3200:	learn: 86.3507599	total: 6m 16s	remaining: 54s
3400:	learn: 86.1141928	total: 6m 41s	remaining: 30.5s
3600:	learn: 85.8766011	total: 7m 5s	remaining: 6.97s


[I 2023-11-09 14:05:03,548] Trial 1 finished with value: 160.40996502421177 and parameters: {'iterations': 3660, 'learning_rate': 0.015871232601449292, 'depth': 4, 'min_data_in_leaf': 10, 'l2_leaf_reg': 5, 'bagging_temperature': 0.8640408928938796, 'random_strength': 0.8141700381950707, 'border_count': 409, 'rsm': 0.9302043034010897}. Best is trial 1 with value: 160.40996502421177.


...

2318:	learn: 79.3319299	total: 4m 55s	remaining: 0us
0:	learn: 260.6976989	total: 125ms	remaining: 5m 33s
200:	learn: 157.1201441	total: 26.1s	remaining: 5m 21s
400:	learn: 115.9882492	total: 53.1s	remaining: 5m 2s
600:	learn: 98.8967754	total: 1m 20s	remaining: 4m 39s
800:	learn: 90.0980549	total: 1m 48s	remaining: 4m 15s
1000:	learn: 85.4917729	total: 2m 17s	remaining: 3m 50s
1200:	learn: 83.0171963	total: 2m 45s	remaining: 3m 24s
1400:	learn: 81.5753088	total: 3m 14s	remaining: 2m 57s
1600:	learn: 80.6154534	total: 3m 42s	remaining: 2m 30s
1800:	learn: 79.6939942	total: 4m 11s	remaining: 2m 2s
2000:	learn: 78.9409437	total: 4m 40s	remaining: 1m 35s
2200:	learn: 78.3109782	total: 5m 9s	remaining: 1m 7s
2400:	learn: 77.6378980	total: 5m 38s	remaining: 39.3s
2600:	learn: 76.8938130	total: 6m 7s	remaining: 11.2s
2679:	learn: 76.6444481	total: 6m 19s	remaining: 0us


[I 2023-11-09 19:01:25,806] Trial 44 finished with value: 152.05085315049257 and parameters: {'iterations': 2680, 'learning_rate': 0.003690520295007481, 'depth': 10, 'min_data_in_leaf': 70, 'l2_leaf_reg': 9, 'bagging_temperature': 0.5059956404021384, 'random_strength': 0.4893692693042781, 'border_count': 901, 'rsm': 0.643652445681666}. Best is trial 41 with value: 151.9035608549706.


0:	learn: 260.3624400	total: 112ms	remaining: 5m 46s
200:	learn: 122.3318972	total: 22.6s	remaining: 5m 26s
400:	learn: 95.2528158	total: 46.3s	remaining: 5m 11s
600:	learn: 88.1690949	total: 1m 10s	remaining: 4m 52s
800:	learn: 85.7791786	total: 1m 34s	remaining: 4m 32s
1000:	learn: 84.3065656	total: 2m	remaining: 4m 11s
1200:	learn: 83.2911681	total: 2m 24s	remaining: 3m 48s
1400:	learn: 82.4107334	total: 2m 49s	remaining: 3m 25s
1600:	learn: 81.6583924	total: 3m 14s	remaining: 3m 1s
1800:	learn: 81.0676957	total: 3m 40s	remaining: 2m 38s
2000:	learn: 80.6672740	total: 4m 6s	remaining: 2m 15s
2200:	learn: 80.1219456	total: 4m 32s	remaining: 1m 51s
2400:	learn: 79.6955123	total: 4m 58s	remaining: 1m 26s
2600:	learn: 79.2564126	total: 5m 25s	remaining: 1m 2s
2800:	learn: 79.0250664	total: 5m 53s	remaining: 37.6s
3000:	learn: 78.7131166	total: 6m 20s	remaining: 12.4s


[I 2023-11-09 19:07:59,579] Trial 45 finished with value: 152.67450933815914 and parameters: {'iterations': 3099, 'learning_rate': 0.007055609190660255, 'depth': 8, 'min_data_in_leaf': 82, 'l2_leaf_reg': 9, 'bagging_temperature': 0.5049087217989455, 'random_strength': 0.5167596614388927, 'border_count': 973, 'rsm': 0.655163154271384}. Best is trial 41 with value: 151.9035608549706.


3098:	learn: 78.4636949	total: 6m 33s	remaining: 0us
0:	learn: 261.0282710	total: 97.7ms	remaining: 4m 30s
200:	learn: 150.7338326	total: 19.8s	remaining: 4m 12s
400:	learn: 111.8128276	total: 40.8s	remaining: 4m


In [ ]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)


with open("optuna-best-parameters_2020:5:00-2020:7:30.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character